sarahfong
# Overview
This notebook details how to set up a *fairly* seamless wynton login/environment

- Here I detail:
    1. how to get setup on wynton 
    2. how to login to wynton *without using your password*
    3. how to set up a virtual environment with miniconda and jupyter notebooks on wynton
    4. how to tunnel from your local computer to wynton

# How to get setup on wynton

## Request a wynton account

website - https://wynton.ucsf.edu/hpc/about/join.html
    
1. select "Request a Wynton Account / Modify an Existing Account (requires UCSF MyAccess login)"
2. login to MyAccess
3. set Nadav as the PI in UCSF PI Email
4. Which email address do I use? 
    - UCSF email account, preferred
    - personal email address, less preferrable. 
5. once you receive your username and password from wynton, login. 

# Login to wynton

In **local terminal**, use your username, and enter: 

- replace "USERNAME" with your username

 **do not type '%%bash'**
   - this command only works inside jupyter notebooks

In [ ]:
%%bash 

ssh USERNAME@log1.wynton.ucsf.edu

## Login to wynton without password
- In the section below, I try to recap steps 1-4 outlined here: https://wynton.ucsf.edu/hpc/howto/log-in-without-pwd.html


- At bottom of page, follow "Connect directly to a development node" steps. 

**THIS WILL SAVE YOU SO MUCH TIME**

1. Setup public-private ssh key, passphrase
- In **local terminal** 
- generate private public ssh key, copy it to wynton

### generate key
- In **local terminal** 

In [ ]:
%%bash

mkdir ./.ssh  # make directory

chmod u=rwx,go= ./.ssh  # change permissions for directory 

ssh-keygen -m PEM -f ./.ssh/laptop_to_wynton  # generate the key

### copy key to wynton
- In **local terminal** 

In [ ]:
%%bash

ssh-copy-id -i ./.ssh/laptop_to_wynton.pub USERNAME@log1.wynton.ucsf.edu  

# this command will prompt login to add key to wynton

### login to wynton using key
- In **local terminal**

In [ ]:
%%bash

ssh -o PreferredAuthentications=publickey,keyboard-interactive -o IdentitiesOnly=yes -i ./.ssh/laptop_to_wynton USERNAME@log1.wynton.ucsf.edu

### set up ssh config, add wynton info
- In **local terminal**

- make config 

In [ ]:
%%bash

touch ./.ssh/config    

### add login info to config
- In **local terminal** 

In [ ]:
%%bash

echo 'Host *.wynton.ucsf.edu' >>./.ssh/config

echo 'User USERNAME' >>./.ssh/config

echo 'IdentityFile ./.ssh/laptop_to_wynton' >>./.ssh/config

### add proxyjump to config to login directly to dev2
- In **local terminal** 

In [ ]:
%%bash

echo 'Host *dev?.wynton.ucsf.edu' >>./.ssh/config
echo 'ProxyJump log1.wynton.ucsf.edu' >>./.ssh/config

### add login alias to .bash_profile 
- In **local terminal**

In [ ]:
%%bash

echo 'alias wynton="ssh dev2.wynton.ucsf.edu"' >>./.bash_profile

### test proxy jump
- In **local terminal** 

In [ ]:
%%bash

wynton

# Set up conda virtual environment and Jupyter Notebooks
- Important for running software packages into your environment without destroying wynton for everyone else. 




## load miniconda
- In **Wynton terminal**
- find latest version by running

In [ ]:
%%bash

ml spider miniconda 

### add shortcut to load miniconda upon login 
- In **Wynton terminal**
- On wynton, **in your home directory**

In [ ]:
%%bash

echo 'if [[ -n "$MODULEPATH" ]]'  >>./.bash_profile
echo 'then'  >>./.bash_profile
echo '    ml load CBI miniconda3-py39/4.12.0' >>./.bash_profile
echo 'fi' >>./.bash_profile

## create virtual environment

- run command below
- **Replace MYENVNAME** to whatever name for your virtual environment
- In **Wynton terminal**

In [ ]:
%%bash

conda create -n MYENVNAME

## add shortcut virtual environment alias to  .bash_profile
- **Replace MYENVNAME** to whatever name for your virtual environment
- In **Wynton terminal**

In [ ]:
%%bash

echo 'alias env="conda activate MYENVNAME"' >>./.bash_profile

## activate your environment
- run command below
- In **Wynton terminal**

In [ ]:
%%bash

env

## download jupyter notebooks 
- In **Wynton terminal**

In [ ]:
%%bash

env  # activate conda environment w alias

conda install -c conda-forge jupyter  # command to install jupyter from anaconda channel (-c)

# tunneling wynton to your local computer

## pick your favorite 4-digit number for the PORTNUMBER
e.g. my favorite is 7778. 

(DO NOT CHOOSE 7778. I WILL FIND YOU.)

## write bash script to launch notebook on wynton

run this bash file whenever you tunnel from wynton to your local computer
- In **Wynton terminal**

In [ ]:
%%bash

touch nb.sh  # make a file to launch your notebook

echo 'source activate MYENVNAME'  >>./nb.sh  # add line to active virtual env before you run your notebook

echo "jupyter notebook --no-browser --ip='*' --port=PORTNUMBER" >>./nb.sh  # add line w favorite portnumber

### add short cut to launch wynton on jupyter
- In **Wynton terminal**

In [ ]:
%%bash

echo 'alias nb="bash ./nb.sh"' >>./.bash_profile  # add shortcut to .bash_profile

## set up tunneling command on your local computer

On **local terminal**

- add function to tunnel to your port on wynton
- you will have to enter the port number manually (see step 4 below)

In [ ]:
%%bash
 
echo 'function wnb { ssh -o PreferredAuthentications=publickey,keyboard-interactive -o IdentitiesOnly=yes -i ~/.ssh/laptop_to_wynton USERNAME@dev2.wynton.ucsf.edu -J USERNAME@log1.wynton.ucsf.edu -L 9999:localhost:"$1"; }'>>./.zprofile  
 
echo 'export -f wnb' >>./.zprofile

## Tunneling, finally

### log into wynton from your local computer
- In **local terminal**

In [ ]:
%%bash

wynton

### on wynton, run jupyter notebook from your favorite port
- In **Wynton terminal** 

In [ ]:
%%bash

nb

### Open up new terminal window on local computer
- In another **local terminal**  (ctrl+ tab) 
- connect to wynton
- change PORTNUMBER to your portnumber from step 3.1

In [ ]:
%%bash
wnb PORTNUMBER

### in web browser, go to web address http://localhost:PORTNUMBER

Remember **both your password and passphrase**

And you did it!